<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE__Transfer_Learning_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import json
import joblib
import datetime
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix

tf.random.set_seed(42)

tf.__version__

In [ ]:
CONFIG = {
    'timestamp'             : str(datetime.datetime.now()),
    'model'                 : 'mobile_net_v2',
    'img_shape'             : (160, 160),
    'spec_config'           : 'LOG_SPEC_W400_H200_HAMM_IMG288',
    'test_fold'             : 'Fold 1',
    'architecture'          : '',
    'batch_size'            : 1024,
    'epochs'                : 0,
    'learning_rate'         : 0.0001,
    'monitor'               : 'val_loss',
    'patience'              : 3,
    'class_weight'          : { 0: 0.71, 1:  2.32, 2: 0.86 },
    'training_time'         : 0,
    'testing_time'          : 0,
    'cm_atick'              : '',
    'cr_atick'              : '',
    'cm_valentina'          : '',
    'cr_valentina'          : ''
}

In [ ]:
BASE_DIR_AF         = '/content/drive/MyDrive/Research/Crowd Emotion v4/'
BASE_DIR_VAL        = '/content/drive/MyDrive/Research/Crowd Emotion Val/'
DATASET_DIR         = 'Dataset_AF/'

LOG_FILE            = '/content/drive/MyDrive/Research/Crowd Emotion Logs/tl_specgrams.txt'
FOLDS               = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
EMOTIONS            = ['Approval', 'Disapproval', 'Neutral']


In [ ]:
CONFIG = {
    'BASE_DIR_AF'      : '/content/drive/MyDrive/Research/Crowd Emotion v4/',
    'BASE_DIR_VAL'     : '/content/drive/MyDrive/Research/Crowd Emotion Val/',
    'DATASET_DIR'      : 'Dataset_AF/',
    'LOG_DIR'          : 'Logs/',
    'FOLDS'            : ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5'],
    'EMOTIONS'         : ['Approval', 'Disapproval', 'Neutral'],

    'MODEL'            : 'mobile_net_v2',
    'IMG_SIZE'         : (160, 160),
    'DATASET_LEN'      : 14882,

    'SPEC_CONFIG'      : 'LOG_SPEC_W400_H200_HAMM_IMG288',

    'TEST_FOLD'        : 'Fold 1',
    'EPOCHS'           : 10,
    'LEARNING_RATE'    : 0.001,
}

In [ ]:
path_AF   = os.path.join(BASE_DIR_AF, DATASET_DIR)
X_AF      = joblib.load(path_AF + CONFIG['spec_config'] + '_X.joblib')
y_AF      = joblib.load(path_AF + CONFIG['spec_config'] + '_y.joblib')
f_AF      = joblib.load(path_AF + CONFIG['spec_config'] + '_f.joblib')
c_AF      = joblib.load(path_AF + CONFIG['spec_config'] + '_f.joblib')

path_VAL  = os.path.join(BASE_DIR_VAL, DATASET_DIR)
test_X    = joblib.load(path_VAL + CONFIG['spec_config'] + '_X.joblib')
test_y    = joblib.load(path_VAL + CONFIG['spec_config'] + '_y.joblib')

mask      = (f == FOLDS.index(CONFIG['test_fold']))
train_X   = X[~mask, :]
train_y   = y[~mask, :]
val_X     = X[mask, :]
val_y     = y[mask, :]

In [ ]:
class TransferLearning():
    def __init__(self, base_model, input_shape, learning_rate, batch_size, epochs):
        self.base_model = base_model
        self.input_shape = input_shape
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        if base_model == 'mobile_net_v2':
            self.base_model = tf.keras.applications.MobileNetV2(
                input_shape     = self.input_shape,
                include_top     = False,
                weights         = 'imagenet'
            )
            self.base_model.trainable = False
            self.preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
            self.model = None
            self.callbacks = None

    def __init_model(self, num_classes):
        inputs = tf.keras.Input(shape=self.input_shape)
        x = self.preprocess(inputs)
        x = self.base_model(x, training=False)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = tf.keras.layers.Dense(num_classes)(x)
        self.model = tf.keras.Model(inputs, outputs)
        self.model.compile(
            optimizer      = tf.keras.optimizers.Adam(lr=self.learning_rate),
            loss           = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics        = ['accuracy']
        )

    def __init_callbacks(self):
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor        = 'val_loss',
            patience       = 3,
            verbose        = 1
        )
        self.callbacks = [
                          early_stopping,
        ]

    def fit(self, train_X, train_y, val_X, val_y):
        num_classes = np.unique(train_y).shape[0]
        self.__init_model(num_classes)
        self.__init_callbacks()
        history = self.model.fit(
            x                 = train_X,
            y                 = train_y,
            batch_size        = self.batch_size,
            epochs            = self.epochs,
            verbose           = 1,
            validation_data   = (val_X, val_y),
            shuffle           = True,
            callbacks         = self.callbacks
        )
    
        return self.model, history 

    def evaluate(self, test_X, test_y):
        prob_model = tf.keras.Sequential([self.model, tf.keras.layers.Softmax()])
        y_pred_hot = prob_model.predict(test_X)
        y_pred = np.argmax(y_pred_hot, axis=1)
        return classification_report(test_y.ravel(), y_pred)

In [ ]:
tl = TransferLearning(
    base_model         = 'mobile_net_v2',
    input_shape        = CONFIG['IMG_SIZE'] + (3,),
    learning_rate      = 0.0001,
    batch_size         = 32,
    epochs             = 100
)
model, history = tl.fit(train_X, train_y, test_X, test_y)
result = tl.evaluate(test_X, test_y)

In [ ]:
result = tl.evaluate(test_X, test_y)

In [ ]:
print(result)